In [29]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [30]:
# !which chromedriver

In [31]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/nadiarichards/.wdm/drivers/chromedriver/mac64/88.0.4324.96]


# NASA Mars News

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8867/nasas-mars-helicopter-reports-in/" target="_self">NASA's Mars Helicopter Reports In </a></div>

In [6]:
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Mars Helicopter Reports In "

In [7]:
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The technology demonstration has phoned home from where it is attached to the belly of NASA’s Perseverance rover. '

In [ ]:
browser.quit()

# NASA Jet Propulsion Laboratory Images

In [46]:
url_images = 'https://www.jpl.nasa.gov/images?query='
browser.visit(url_images)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59130): Max retries exceeded with url: /session/c7dc1c98afe54b0bba316c8bfec504f5/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc086200d90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [47]:
html = browser.html
img_soup = soup(html, 'html.parser')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59130): Max retries exceeded with url: /session/c7dc1c98afe54b0bba316c8bfec504f5/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc08569cd30>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [48]:
browser.click_link_by_href("/images/mastcam-z-looks-at-its-calibration-target")

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59130): Max retries exceeded with url: /session/c7dc1c98afe54b0bba316c8bfec504f5/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc082a1bca0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [42]:
images = img_soup.find('div', class_='BaseImage  object-scale-down')
images

In [45]:
browser.quit()

In [ ]:
images = img_soup.find('div', class_='sm:object-cover object-cover')
images

In [39]:
browserfind_by_tag("class_="SearchResultCard-image xl:pl-14 sm:col-span-3 md:col-span-3 lg:col-span-2 xl:col-span-2 col-span-3").first

SyntaxError: invalid syntax (<ipython-input-39-98956a3b97da>, line 1)

In [ ]:
text-subtitle lg:block hidden mb-3

In [ ]:
sidebar = soup.find('div', id_='96634')

categories = sidebar.find_all('li')

category_list = []
url_list = []
book_url_list = []
for category in categories:
    title = category.text.strip()
    category_list.append(title)
    book_url = category.find('a')['href']
    url_list.append(book_url)

book_url_list = ['http://books.toscrape.com/' + url for url in url_list]

titles_and_urls = zip(category_list, book_url_list)

try:
    for title_url in titles_and_urls:
        browser.links.find_by_partial_text('next').click()
except ElementDoesNotExist:
    print("Scraping Complete")

In [ ]:
for x in range(50):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
    articles = soup.find_all('article', class_='product_pod')

    # Iterate through each book
    for article in articles:
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
        h3 = article.find('h3')
        link = h3.find('a')
        href = link['href']
        title = link['title']
        print('-----------')
        print(title)
        print('http://books.toscrape.com/' + href)

    # Click the 'Next' button on each page
    try:
        browser.links.find_by_partial_text('next').click()
          
    except:
        print("Scraping Complete")

In [ ]:
img_soup.prettify()

In [ ]:
img_elem.find("div", class_='content_title')

In [ ]:
img_url_rel = img_soup.select_one('img.BaseImage object-contain')
img_url_rel

In [ ]:
.get("src")

In [9]:
browser.quit()

# Mars Facts Table

In [10]:
mars_url="https://space-facts.com/mars/"

In [11]:
tables = pd.read_html(mars_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [15]:
df=tables[1]
df.head()

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


In [25]:
html_table=df.to_html()
html_table 

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemispheres

In [49]:
url_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hemispheres)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59130): Max retries exceeded with url: /session/c7dc1c98afe54b0bba316c8bfec504f5/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc086216100>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
html = browser.html
hemisphere_soup = soup(html, 'html.parser')

In [ ]:
sidebar = soup.find('ul', class_='nav-list')

categories = sidebar.find_all('li')

category_list = []
url_list = []
book_url_list = []
for category in categories:
    title = category.text.strip()
    category_list.append(title)
    book_url = category.find('a')['href']
    url_list.append(book_url)

book_url_list = ['http://books.toscrape.com/' + url for url in url_list]

titles_and_urls = zip(category_list, book_url_list)

try:
    for title_url in titles_and_urls:
        browser.links.find_by_partial_text('next').click()
except ElementDoesNotExist:
    print("Scraping Complete")